In [1]:
import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry

requests.adapters.DEFAULT_POOLSIZE = 100
retries = Retry(total=20)
# 세션을 생성하고 ConnectionPool 크기를 설정
session = requests.Session()
session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))

import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry


# soup = BeautifulSoup(html, 'html.parser')
# items = soup.find_all('option')

In [15]:
def extract_soup(url):
    headers = {
        "accept": "application/json, text/javascript, */*; q=0.01",
        "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "sec-ch-ua": "\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"macOS\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "x-requested-with": "XMLHttpRequest",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    response = session.get(url, headers=headers)
    response.raise_for_status()
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        raise Exception(f"[{response.status_code}] {response.text}")


In [51]:
result_list = []

In [52]:

def extract_제조사별_list(soup):
    for div in soup.find_all('div', style="font-size:14px;color:#336600;font-weight:bold;text-decoration:underline"):
        a = div.find('a')
        if a:
            result_list.append('https://www.sdfi.co.kr/S/' + a.get('href'))


def extract_식품별_list(soup):
    for div in soup.find_all('li', style="margin-bottom:12px"):
        a = div.find('a')
        if a:
            result_list.append('https://www.sdfi.co.kr/S/' + a.get('href'))




In [53]:
import time

# 'https://www.sdfi.co.kr/S/CompanyList.asp'
# https://www.sdfi.co.kr/S/CompanyList.asp?page=2&search=&keyword=
# 2 ~ 1512
# soup = extract_soup('https://www.sdfi.co.kr/S/CompanyList.asp')
# extract_list(soup)

#가공 (417)  :https://www.sdfi.co.kr/S/TypeList.asp?search=cls_no&keyword=2

target_list = [
    (2, 417), #가공
    (5, 285), #건강기능
    (1, 807), #농임축수산물
    (3, 625), #첨가물
]

fail_list = []
for keywords, last_page in target_list:
    for i in range(1, last_page + 1):
        try:
            soup = extract_soup(f"https://www.sdfi.co.kr/S/TypeList.asp?page={i}&search=cls_no&keyword={keywords}")
            extract_식품별_list(soup)
            time.sleep(0.3)
        except Exception as e:
            fail_list.append({
                "page": i,
                "error": str(e)
            })
            print(fail_list[-1])

In [54]:
import json
with open('./output/sdfi_product_link_list.json', '+wt') as fp:
    json.dump(result_list, fp)

세부 페이지 수집

In [93]:
from datetime import datetime
import time

detail_list = []

for i, link in enumerate(result_list):
    if i <= 120:
        continue
    
    page_no = 1
    print(link)
    while True:
        
        sub_url = link + f"&page={page_no}"
        time.sleep(0.3)
        soup = extract_soup(sub_url)

        large_category = None
        small_category = None
        path = soup.find('div', style="text-align:left;border-bottom:1px;padding-bottom:6px; border-bottom-style:dotted;border-bottom-width: 1px;border-bottom-color: #C0C0C0;")
        if path:
            path_list = path.get_text(strip=True).split('>')
            large_category = path_list[2]
            small_category = path_list[3]

        table = soup.find('table')
        li_list = table.find_all('li')

        for li in li_list:

            item = {
                "url": sub_url,
                "large_category": large_category,
                "small_category": small_category
            }

            if li.find('a'):
                item["product_ko"] = li.find('a').text
            if li.find('div', style="font-size:9px;margin-top:4px"):
                item["product_en"] = li.find('div', style="font-size:9px;margin-top:4px").text

            detail = li.find('div', style="margin-bottom:28px;font-size:11px")
            if detail:
                for div in detail.find_all('div'):
                    txt = div.text.strip()
                    # if txt.startswith('분류명'):
                    #     print(txt.replace('분류명 :', '').strip())
                    if txt.startswith('제조사'):
                        item["company_name"] = txt.replace('제조사 :', '').strip()
                    elif txt.startswith('수입사'):
                        item["buyer"] = txt.replace('수입사 :', '').strip()
                    elif txt.startswith('원산지'):
                        item["origin_country"] = txt.replace('원산지 :', '').strip()
            detail_list.append(item)
        
        if not li_list or len(li_list) < 30: # 없거나 max 못채우면
            break
        else:
            page_no += 1
        
    if len(detail_list) > 10000:
        df = pd.DataFrame(detail_list)
        df.to_parquet(f'./output/sdfi/detail_result_{datetime.now().strftime("%Y%m%d_%H%M%S")}.parquet')
        detail_list = []



https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=929
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=930
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=931
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=932
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=933
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=934
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=935
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=936
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=937
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=938
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=939
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=940
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=941
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword=942
https://www.sdfi.co.kr/S/ProductList.asp?search=typ_no&keyword

In [95]:
df

,url,large_category,small_category,product_ko,product_en,company_name,buyer,origin_country
0,https://www.sdfi.co.kr/S/ProductList.asp?searc...,식품첨가물,홍국적색소,천연색소레드(M),NATURAL COLOR M,"DAIWA DYESTUFFMFG CO.,LTD",켐코코리아 주식회사,일본
1,https://www.sdfi.co.kr/S/ProductList.asp?searc...,식품첨가물,홍국적색소,홍국적색소,MONASCUS RED POWDER,"HUBEI ZIXIN BIOLOGICAL TECHNOLOGY CO.,LTD,",주식회사 남영F&C,중국
2,https://www.sdfi.co.kr/S/ProductList.asp?searc...,식품첨가물,홍국적색소,홍국적색소,MONASCUS RED POWDER,"WUHAN GREEN FOOD BIOLOGICAL ENGINEERING CO., LTD.",(주)남영F&C,중국
3,https://www.sdfi.co.kr/S/ProductList.asp?searc...,식품첨가물,홍국적색소,홍국색소,MONASRED(MONASCUS COLOR),SINOCHEM QINGDAO CO LTD,지연,중국
4,https://www.sdfi.co.kr/S/ProductList.asp?searc...,식품첨가물,홍국적색소,홍국적색소,MONASCUS RED,"QINGDAO PENGYUAN KANGHUA NATURAL SOURCE CO.,LTD.",(주)두비산업,중국
...,...,...,...,...,...,...,...,...
1170,https://www.sdfi.co.kr/S/ProductList.asp?searc...,식품첨가물,L-히스티딘,L-히스티딘,L-HISTIDINE,"SHANGHAI KYOWA AMINO ACID., CO.LTD.",(주)네오크레마,중국
1171,https://www.sdfi.co.kr/S/ProductList.asp?searc...,식품첨가물,L-히스티딘,L-히스티딘,L-HISTIDINE,WUHAN AMINO ACID BIO-CHEMICAL CO LTD,화일약품(주),중국
1172,https://www.sdfi.co.kr/S/ProductList.asp?searc...,식품첨가물,Mace,메이스 오일 IN11737,MACE OIL IN11737,LIONEL HITCHEN (ESSENTIAL OILS) LTD.,주식회사 동방푸드마스타,영국
1173,https://www.sdfi.co.kr/S/ProductList.asp?searc...,식품첨가물,Peppermint,페파민트 오일(100% 천연),PEPPERMINT OIL (100% NATURAL),ERNESTO VENTOS SA,화인코리아코퍼레이션(주),스페인


In [88]:
df.to_parquet(f'./output/sdfi/detail_result_{datetime.now().strftime("%Y%m%d_%H%M%S")}.parquet')

In [92]:
print(i, page_no)

120 83


In [ ]:
table = soup.find('table')
title = table.find('div', style="font-size:20px;margin-left:30px;font-weight:bold")
if title:
    name = title.get_text(strip=True)
    print(name)
    li_list = table.find_all('li')

    for li in li_list:

        if li.find('a'):
            product_ko = li.find('a').text
        if li.find('div', style="font-size:9px;margin-top:4px"):
            product_en = li.find('div', style="font-size:9px;margin-top:4px").text

        detail = li.find('div', style="margin-bottom:28px;font-size:11px")
        if detail:
            for div in detail.find_all('div'):
                txt = div.text.strip()
                if txt.startswith('분류명'):
                    print(txt.replace('분류명 :', '').strip())
                elif txt.startswith('수입사'):
                    print(txt.replace('수입사 :', '').strip())
                elif txt.startswith('원산지'):
                    print(txt.replace('원산지 :', '').strip())


In [98]:
from google.cloud import storage, bigquery

storage_client = storage.Client(project="grainscanner")
biguqery_client = bigquery.Client(project="grainscanner")

def _parquet_import_bigquery(project:str, dataset:str, table_name:str, bucket_name:str, gcs_path:str) -> None:
    print('_____________________________________________________________________________________\n')
    uri = f"gs://{bucket_name}/{gcs_path}"
    

    bucket = storage.Bucket(storage_client, bucket_name)
    blobs = bucket.list_blobs(prefix=uri.split(bucket_name)[1].strip('/').strip('*'))
    
    if any(blob.name for blob in blobs):
        print(f'BigQuery Table Insert...')

        job_config = bigquery.LoadJobConfig()
        job_config.source_format = bigquery.SourceFormat.PARQUET  # Parquet 파일 포맷으로 설정

        load_job = biguqery_client.load_table_from_uri(uri, f"{project}.{dataset}.{table_name}", job_config=job_config)
        load_job.result()  # Waits for the job to complete.

        destination_table = biguqery_client.get_table(f'{project}.{dataset}.{table_name}')  # Make an API request.
        print("Loaded {} rows.".format(destination_table.num_rows))
    else:
        print(f'No Files - {uri}')
        


_parquet_import_bigquery("grainscanner", "external_ingestion", "unipass_foreign_customer_raw", "greenlabs-data-grainscanner", "crawling/temp/unipass/*")


_____________________________________________________________________________________

BigQuery Table Insert...
Loaded 2109346 rows.
